In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.3 MB/s eta 0:00:00


In [18]:
import json
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, FileLink, clear_output
from faker import Faker
import random
from google.colab import files

# Faker initialization in Spanish
fake = Faker("es_ES")

# Real spanish addresses
postal_codes_spain = {
    "Madrid": {"postalCode": "28001", "streets": ["Gran Vía", "Calle de Alcalá", "Paseo del Prado", "Calle Serrano"]},
    "Barcelona": {"postalCode": "08001", "streets": ["La Rambla", "Passeig de Gràcia", "Carrer de Balmes", "Avinguda Diagonal"]},
    "Valencia": {"postalCode": "46001", "streets": ["Calle de Colón", "Avenida del Puerto", "Calle de la Paz", "Plaza del Ayuntamiento"]},
    "Sevilla": {"postalCode": "41001", "streets": ["Avenida de la Constitución", "Calle Sierpes", "Calle Betis", "Calle Alfonso XII"]},
    "Zaragoza": {"postalCode": "50001", "streets": ["Paseo de la Independencia", "Calle del Coso", "Avenida de Goya", "Calle San Vicente de Paúl"]},
    "Málaga": {"postalCode": "29001", "streets": ["Calle Larios", "Paseo del Parque", "Calle Granada", "Avenida de Andalucía"]},
    "Murcia": {"postalCode": "30001", "streets": ["Gran Vía Alfonso X", "Calle Trapería", "Avenida de la Libertad", "Calle Platería"]},
    "Palma de Mallorca": {"postalCode": "07001", "streets": ["Passeig del Born", "Avinguda Jaume III", "Calle Sindicat", "Calle de San Miguel"]},
    "Las Palmas de Gran Canaria": {"postalCode": "35001", "streets": ["Calle Triana", "Avenida Mesa y López", "Calle León y Castillo", "Calle Tomás Morales"]},
    "Bilbao": {"postalCode": "48001", "streets": ["Gran Vía Don Diego López de Haro", "Calle Ercilla", "Calle Licenciado Poza", "Avenida Sabino Arana"]}
}

# Function to generate a spanish correct address
def generate_real_spanish_address():
    city, details = random.choice(list(postal_codes_spain.items()))
    return {
        "use": "home",
        "line": [random.choice(details["streets"])],
        "city": city,
        "postalCode": details["postalCode"],
        "country": "España"
    }

# Function to generate a random address
def generate_random_address():
    return {
        "use": "home",
        "line": [fake.street_address()],
        "city": fake.city(),
        "postalCode": fake.postcode(),
        "country": fake.country()
    }

# Define the available FHIR resources and their fields
fhir_resources = {
    "Patient": {
        "identifier": lambda: [{"use": "official", "system": "https://hospital.example.com/patients", "value": fake.uuid4()}],
      "active": lambda: random.choice([True, False]),
      "name": lambda: [{"use": "official", "family": fake.last_name(), "given": [fake.first_name()]}],
      "telecom": lambda: [{"system": "phone", "value": fake.phone_number(), "use": "mobile"}],
      "gender": lambda: random.choice(["male", "female", "other", "unknown"]),
      "birthDate": lambda: fake.date_of_birth(minimum_age=0, maximum_age=100).isoformat(),
      "deceasedBoolean": lambda: random.choice([True, False]) if random.random() > 0.1 else None,
      "deceasedDateTime": lambda: fake.date_time_this_century().isoformat() if random.random() > 0.1 else None,
      "address": lambda: [generate_real_spanish_address()] if real_address_checkbox.value else [generate_random_address()],
      "maritalStatus": lambda: {"text": random.choice(["Married", "Single", "Divorced", "Widowed"])},
      "multipleBirthBoolean": lambda: random.choice([True, False]),
      "multipleBirthInteger": lambda: random.randint(1, 5) if random.random() > 0.5 else None,
      "photo": lambda: [{"contentType": "image/jpeg", "url": "https://example.com/photos/" + fake.uuid4() + ".jpg"}],
      "contact": lambda: [{
          "relationship": [{"text": random.choice(["Parent", "Sibling", "Friend", "Partner"])}],
          "name": {"family": fake.last_name(), "given": [fake.first_name()]},
          "telecom": [{"system": "phone", "value": fake.phone_number(), "use": "home"}],
          "address": [generate_real_spanish_address()] if real_address_checkbox.value else [generate_random_address()],
          "gender": random.choice(["male", "female", "other", "unknown"]),
          "organization": {"reference": "Organization/" + fake.uuid4()},
          "period": {"start": fake.date_time_this_century().isoformat()}
      }],
      "communication": lambda: [{
          "language": {"text": random.choice(["English", "Spanish", "French", "German"])},
          "preferred": random.choice([True, False])
      }],
      "generalPractitioner": lambda: [{"reference": "Practitioner/" + fake.uuid4()}],
      "managingOrganization": lambda: {"reference": "Organization/" + fake.uuid4()},
      "link": lambda: [{
          "other": {"reference": "Patient/" + fake.uuid4()},
          "type": random.choice(["replaced-by", "replaces", "refer", "seealso"])
      }]
    },
    "Condition": {
        "identifier": lambda: [{"use": "official", "system": "https://hospital.example.com/conditions", "value": fake.uuid4()}],
        "clinicalStatus": lambda: {"text": random.choice(["active", "recurrence", "remission", "resolved"])},
        "verificationStatus": lambda: {"text": random.choice(["unconfirmed", "confirmed", "differential"])},
        "category": lambda: [{"text": random.choice(["problem-list-item", "encounter-diagnosis"])}],
        "code": lambda: {"text": random.choice(["Hypertension", "Diabetes", "Asthma", "Migraine"])},
        "subject": lambda: {"reference": "Patient/" + fake.uuid4()},
        "onsetDateTime": lambda: fake.date_time_this_century().isoformat()
    },
    "Observation": {
        "identifier": lambda: [{"use": "official", "system": "https://hospital.example.com/observations", "value": fake.uuid4()}],
        "status": lambda: random.choice(["final", "amended", "cancelled"]),
        "category": lambda: [{"text": "vital-signs"}],
        "code": lambda: {"text": random.choice(["Blood Pressure", "Heart Rate", "Temperature"])},
        "subject": lambda: {"reference": "Patient/" + fake.uuid4()},
        "effectiveDateTime": lambda: fake.date_time_this_century().isoformat(),
        "valueQuantity": lambda: {"value": round(random.uniform(80, 180), 1), "unit": "mmHg"}
    },
    "Encounter": {
        "identifier": lambda: [{"use": "official", "system": "https://hospital.example.com/encounters", "value": fake.uuid4()}],
        "status": lambda: random.choice(["planned", "in-progress", "on-hold", "finished", "cancelled"]),
        "class": lambda: {"text": random.choice(["inpatient", "outpatient", "ambulatory"])},
        "subject": lambda: {"reference": "Patient/" + fake.uuid4()},
        "participant": lambda: [{"individual": {"reference": "Practitioner/" + fake.uuid4()}}],
        "period": lambda: {
            "start": fake.date_time_this_century().isoformat(),
            "end": fake.date_time_this_century().isoformat()
        }
    }
}

# Widgets to select the resource type and the amount of them
resource_selection_widget = widgets.Dropdown(
    options=list(fhir_resources.keys()),
    description="Recurso FHIR"
)
num_resources_widget = widgets.IntSlider(value=5, min=1, max=100, step=1, description="N° Recursos")

# Container for the fields selection checkboxes
field_checkboxes = {}

# Function to update the chackboxes when another resource is selected
def update_field_checkboxes(*args):
    global field_checkboxes
    selected_resource = resource_selection_widget.value

    # Clear previous checkboxes
    field_checkboxes.clear()

    # Create new checkboxes for the fields of the resource selected
    for field in fhir_resources[selected_resource].keys():
        field_checkboxes[field] = widgets.Checkbox(value=True, description=field)

    # Show the checkboxes in a vertical box
    display_fields()

# Function to show the checkboxes in the interface
def display_fields():
    field_box.children = list(field_checkboxes.values())

# Widget for the field checkboxes
field_box = widgets.VBox()

# Call the initial update of the checkboxes
update_field_checkboxes()

# Button to generate the resources
generate_button = widgets.Button(
    description="Generar Recursos",
    button_style="primary",
    tooltip="Generar recursos FHIR",
    icon="plus"
)

# Button to download the JSON
download_button = widgets.Button(
    description="Descargar JSON",
    button_style="info",
    tooltip="Descargar archivo JSON",
    icon="download",
    disabled=True
)

# Checkbox to select real spanish addresses or not
real_address_checkbox = widgets.Checkbox(
    value=False,
    description="Generar direcciones realistas de España",
    indent=False
)

output_area = widgets.Output()

# Function to generate the selected FHIR resources
def generate_fhir_resources(num_resources, selected_fields, resource_type):
    resources = []
    for _ in range(num_resources):
        resource = {field: fhir_resources[resource_type][field]() for field in selected_fields}
        resource["resourceType"] = resource_type
        resources.append(resource)
    return resources

# Function to show the result and export JSON
def on_generate_button_clicked(b):
  with output_area:
    clear_output()
    num_resources = num_resources_widget.value
    selected_resource = resource_selection_widget.value

    # Obtain the selected fields
    selected_fields = [field for field, checkbox in field_checkboxes.items() if checkbox.value]

    resources = generate_fhir_resources(num_resources, selected_fields, selected_resource)

    # DataFrame conversion to visualization
    df = pd.DataFrame(resources)
    display(df)

    # Store in JSON file
    json_filename = f"fhir_{selected_resource.lower()}s.json"
    with open(json_filename, "w") as f:
        json.dump(resources, f, indent=4)

    download_button.disabled = False
    download_button.tooltip = f"Descargar {json_filename}"
    download_button.on_click(lambda _: download_json(json_filename))

    display(download_button)

    print(f"📂 Archivo JSON generado: {json_filename}")

# Function to download the JSON file
def download_json(filename):
    files.download(filename)

# Asign the function to the button
generate_button.on_click(on_generate_button_clicked)

# Connect the fields update with the resource change
resource_selection_widget.observe(update_field_checkboxes, names="value")

# Show the interface in Google Colab
display(resource_selection_widget, num_resources_widget, field_box, real_address_checkbox, generate_button, output_area)

Dropdown(description='Recurso FHIR', options=('Patient', 'Condition', 'Observation', 'Encounter'), value='Pati…

IntSlider(value=5, description='N° Recursos', min=1)

Checkbox(value=False, description='Generar direcciones realistas de España', indent=False)

Button(button_style='primary', description='Generar Recursos', icon='plus', style=ButtonStyle(), tooltip='Gene…

Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>